# Übersicht über die Datensätze

In dieser Arbeit wurden zwei Kaggle-Datensätze zur Maskenerkennung verwendet. Der erste Datensatz namens "Face Mask Detection Dataset", steht unter [diesem Link](https://www.kaggle.com/datasets/wobotintelligence/face-mask-detection-dataset) zur Verfügung und enthält JSON-Annotationen. Der zweite Datensatz, "Kaggle Face Mask Detection Full", kann unter [diesem Link](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection) gefunden werden. Dieser Datensatz zeichnet sich durch XML-Annotationen aus und wurde durch gezielte Augmentation erweitert, um Bilder mit schwierigen Lichtverhältnissen zu simulieren und einen Datensatz für nächtliche Bedingungen zu generieren.

## Klassen im nicht-dunklen Datensatz (20 Klassen):
| Klassenindex | Klasse                  |
|--------------|-------------------------|
| 0            | face_no_mask            |
| 1            | face_with_mask          |
| 2            | mask_surgical           |
| 3            | hat                     |
| 4            | eyeglasses              |
| 5            | face_other_covering     |
| 6            | face_with_mask_incorrect|
| 7            | mask_colorful           |
| 8            | helmet                  |
| 9            | sunglasses              |
| 10           | scarf_bandana           |
| 11           | hair_net                |
| 12           | goggles                 |
| 13           | face_shield             |
| 14           | hijab_niqab             |
| 15           | turban                  |
| 16           | balaclava_ski_mask      |
| 17           | gas_mask                |
| 18           | hood                    |
| 19           | other                   |

## Klassen im dunklen Datensatz (3 Klassen):
| Klassenindex | Klasse                  |
|--------------|-------------------------|
| 0            | without_mask            |
| 1            | with_mask               |
| 2            | mask_weared_incorrect   |

# Struktur der zu generierenden Datensätze für YOLO-Training

Die Datenvorbereitung umfasst die Konvertierung und Aufbereitung der Datensätze für das YOLO-Training. Die generierten Datensätze werden im Verzeichnis "YOLO/dataset" strukturiert abgelegt und umfassen folgende Untergruppen:

1. **all_classes:** Enthält alle Klassen und wird beim Training auf dem nicht-dunklen Datensatz verwendet.
2. **face_classes:** Berücksichtigt nur die Gesichtsklassen von den insgesamt 20 Klassen während des Trainings auf dem nicht-dunklen Datensatz.
3. **single_person:** Bezieht sich ausschließlich auf die Gesichtsklassen und einzelne Personen beim Training auf dem nicht-dunklen Datensatz.
4. **dark_dataset:** Umfasst alle Klassen und wird beim Training auf dem dunklen Datensatz verwendet.


# Konvertierung ins YOLO-Format
Für das YOLO-Training müssen Annotationen ins YOLO-Format (txt) konvertiert werden. Die Konvertierung von JSON-Daten folgt diesen Regeln:

- Erste Spalte: Klassenindizes
- Zweite bis fünfte Spalte: Koordinaten der Bounding Box (x, y, w, h), diese müssen normalisiert werden.

In [37]:
import os
import json
import shutil
import cv2
import zipfile
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

### Erstellung des All_Classes-Datensatzes für das Training

In diesem Abschnitt werden alle 20 Klassen für das Training berücksichtigt. Der Datensatz wird für das YOLO-Training in 80% Trainings- und 20% Validierungsdatensatz aufgeteilt. Die Annotationen im Datensatz sind im JSON-Format gespeichert und müssen für das YOLO-Training noch ins YOLO-Format umgewandelt werden.

In [38]:
# Funktion zur Überprüfung, ob eine Datei Face-Klassen enthält
def contains_face_classes(json_path, class_mapping):
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    for bbox in json_data["Annotations"]:
        class_name = bbox["classname"]
        if class_name in class_mapping:
            return True
        
    return False

In [39]:
# Funktion zur Konvertierung der Labels ins YOLOv5-Format
def convert_labels_to_yolov5_format(json_path, image_path, output_dir, class_mapping):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Erstelle YOLOv5-Annotationen
    with open(json_path, 'r') as f:
        json_data = json.load(f)

    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    yolo_annotations = []
    for bbox in json_data["Annotations"]:
        x_min, y_min, x_max, y_max = bbox["BoundingBox"]
        class_name = bbox["classname"]

        if class_name not in class_mapping:
            continue  # Überspringe nicht relevante Klassen

        # Klassenindizierung
        class_index = class_mapping[class_name]

        # Normalisierung der Koordinaten
        x_center = (x_min + x_max) / (2 * image_width)
        y_center = (y_min + y_max) / (2 * image_height)
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        yolo_annotation = f"{class_index} {x_center} {y_center} {width} {height}"
        yolo_annotations.append(yolo_annotation)

    image_filename = os.path.splitext(os.path.basename(image_path))[0]
    output_filename = f"{image_filename}.txt"
    output_path = os.path.join(output_dir, output_filename)

    # Erstelle YOLOv5-Annotationen
    with open(output_path, 'w') as f:
        for annotation in yolo_annotations:
            f.write(annotation + '\n')

In [40]:
# Pfade zu den Daten
json_dir = "../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/annotations"
image_dir = "../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images"
dataset_output_dir = "dataset/all_classes"

# Klassen Mapping
class_mapping_all = {
    "face_no_mask": 0,
    "face_with_mask": 1,
    "mask_surgical": 2,
    "hat": 3,
    "eyeglasses": 4,
    "face_other_covering": 5,
    "face_with_mask_incorrect": 6,
    "mask_colorful": 7,
    "helmet": 8,
    "sunglasses": 9,
    "scarf_bandana": 10,
    "hair_net": 11,
    "goggles": 12,
    "face_shield": 13,
    "hijab_niqab": 14,
    "turban": 15,
    "balaclava_ski_mask": 16,
    "gas_mask": 17,
    "hood": 18,
    "other": 19
}


# Bilder und Annotationen einlesen
images = [os.path.join(image_dir, x) for x in os.listdir(image_dir)]
annotations = [os.path.join(json_dir, x) for x in os.listdir(json_dir) if x.endswith(".json")]

# Train-Test-Split
if len(images) < 2 or len(annotations) < 2:
    print("Nicht genügend Datenpunkte für den Split vorhanden.")
    exit()

train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.3, random_state=1)
val_images, test_images, val_annotations, test_annotations = train_test_split(test_images, test_annotations, test_size=0.5, random_state=1)

# Anwendung auf Trainingsdaten
for image_path, json_path in zip(train_images, train_annotations):
    convert_labels_to_yolov5_format(json_path, image_path, os.path.join(dataset_output_dir, 'labels', 'train'), class_mapping_all)

# Anwendung auf Validierungsdaten
for image_path, json_path in zip(val_images, val_annotations):
    convert_labels_to_yolov5_format(json_path, image_path, os.path.join(dataset_output_dir, 'labels', 'val'), class_mapping_all)

# Kopiere Bilder in die entsprechenden Ordner
for split, split_images in zip(['train', 'val'], [train_images, val_images]):
    split_images_output_dir = os.path.join(dataset_output_dir, 'images', split)
    os.makedirs(split_images_output_dir, exist_ok=True)
    for image_path in split_images:
        image_filename = os.path.basename(image_path)
        shutil.copy(image_path, os.path.join(split_images_output_dir, image_filename))

# Erstelle classes.txt-Datei, mit allen Klassen und deren Indizierung
for split in ['train', 'val']:
    classes_file_path = os.path.join(dataset_output_dir, 'labels', split, 'classes.txt')
    with open(classes_file_path, 'w') as f:
        for class_name, class_index in class_mapping_all.items():
            f.write(f"{class_index} {class_name}\n")

### Erstellung des Face_Classes-Datensatzes für das Training

In diesem Abschnitt werden nur die 4 Face-Klassen (siehe unten) aus dem nicht-dunklen Datensatz betrachtet. Hierfür werden nicht relevante Bilder und Labels im Datensatz identifiziert und gelöscht. Anschließend erfolgt die Aufteilung der Daten in 80% Training und 20% Validierungsdatensatz.

| Klassenindex | Klasse                  |
|--------------|-------------------------|
| 0            | face_no_mask            |
| 1            | face_with_mask          |
| 2            | face_other_covering     |
| 3            | face_with_mask_incorrect|

In [41]:
# Funktion zur Entfernung leerer Dateien
def remove_empty_files(json_dir, images_dir, class_mapping):
    for json_filename in os.listdir(json_dir):
        if json_filename.endswith(".json"):
            json_path = os.path.join(json_dir, json_filename)
            image_name, _ = os.path.splitext(json_filename)  # Extract image name without the ".json" extension

            # Construct the image path
            image_path = os.path.join(images_dir, image_name)

            # Check if the .json file is empty or does not contain face classes
            if os.path.getsize(json_path) == 0 or not contains_face_classes(json_path, class_mapping):
                print(f"Delete Labeldata: {json_path}")
                os.remove(json_path)

                # Delete the corresponding picture if available
                if os.path.exists(image_path):
                    print(f"Delete the corresponding picture: {image_path}")
                    os.remove(image_path)

In [42]:
# Pfade zu den Daten
dataset_output_dir = "dataset/face_classes"

# Klassen Mapping
relevant_classes = ["face_no_mask", "face_with_mask", "face_other_covering", "face_with_mask_incorrect"]
class_mapping = {class_name: idx for idx, class_name in enumerate(relevant_classes)}

# Entferne leere und nicht relevante Dateien
remove_empty_files(json_dir, image_dir, class_mapping)

# Bilder und Annotationen einlesen
images = [os.path.join(image_dir, x) for x in os.listdir(image_dir)]
annotations = [os.path.join(json_dir, x) for x in os.listdir(json_dir) if x.endswith(".json")]

# Train-Test-Split
if len(images) < 2 or len(annotations) < 2:
    print("Nicht genügend Datenpunkte für den Split vorhanden.")
    exit()

train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.3, random_state=1)
val_images, test_images, val_annotations, test_annotations = train_test_split(test_images, test_annotations, test_size=0.5, random_state=1)

# Anwendung auf Trainingsdaten
for image_path, json_path in zip(train_images, train_annotations):
    convert_labels_to_yolov5_format(json_path, image_path, os.path.join(dataset_output_dir, 'labels', 'train'), class_mapping)

# Anwendung auf Validierungsdaten
for image_path, json_path in zip(val_images, val_annotations):
    convert_labels_to_yolov5_format(json_path, image_path, os.path.join(dataset_output_dir, 'labels', 'val'), class_mapping)

# Kopiere Bilder in die entsprechenden Ordner
for split, split_images in zip(['train', 'val'], [train_images, val_images]):
    split_images_output_dir = os.path.join(dataset_output_dir, 'images', split)
    os.makedirs(split_images_output_dir, exist_ok=True)
    for image_path in split_images:
        image_filename = os.path.basename(image_path)
        shutil.copy(image_path, os.path.join(split_images_output_dir, image_filename))

# Erstelle classes.txt-Datei, mit allen Klassen und deren Indizierung
for split in ['train', 'val']:
    classes_file_path = os.path.join(dataset_output_dir, 'labels', split, 'classes.txt')
    with open(classes_file_path, 'w') as f:
        for class_name, class_index in class_mapping.items():
            f.write(f"{class_index} {class_name}\n")

Delete Labeldata: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/annotations\1855.jpg.json
Delete the corresponding picture: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images\1855.jpg
Delete Labeldata: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/annotations\1945.jpg.json
Delete the corresponding picture: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images\1945.jpg
Delete Labeldata: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/annotations\2135.jpg.json
Delete the corresponding picture: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images\2135.jpg
Delete Labeldata: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/annotations\3102.png.json
Delete the corresponding picture: ../Datasets/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images\3

### Einzel-Personen-Datensatz

In diesem Datensatz wird ebenfalls der nicht-dunkle Datensatz mit den JSON-Annotationen verarbeitet. Der Fokus liegt hier ausschließlich auf einzelnen Gesichtern. Dabei werden nur die 4 Face_Classes berücksichtigt. Es ist bekannt, dass in einer TXT-Datei mehrere Face_Classes darauf hinweisen, dass mehrere Personen erkannt wurden. Solche TXT-Dateien und die dazugehörigen Bilder werden dann in den Ordner "single_person" kopiert.

In [43]:
# Filtert die Einzelpersonen vor dem Split
def filter_single_persons(labels_dir, images_dir, output_labels_dir, output_images_dir):
    if not os.path.exists(output_labels_dir):
        os.makedirs(output_labels_dir)
    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    for split in ['train', 'val']:
        split_labels_dir = os.path.join(labels_dir, split)
        split_images_dir = os.path.join(images_dir, split)

        for filename in os.listdir(split_labels_dir):
            if filename.endswith(".txt"):
                txt_path = os.path.join(split_labels_dir, filename)
                image_name, _ = os.path.splitext(filename)

                # Lese die Anzahl der Personen aus der .txt-Datei
                with open(txt_path, 'r') as f:
                    num_persons = sum(1 for line in f)

                # Wenn nur eine Person vorhanden ist, kopiere die .txt-Datei und das Bild
                if num_persons == 1:
                    output_txt_path = os.path.join(output_labels_dir, filename)

                    # Suche nach dem Bild mit verschiedenen Erweiterungen (.jpg, .jpeg, .png)
                    image_extensions = [".jpg", ".jpeg", ".png"]
                    for ext in image_extensions:
                        image_path = os.path.join(split_images_dir, f"{image_name}{ext}")
                        if os.path.exists(image_path):
                            break

                    if os.path.exists(image_path):
                        output_image_path = os.path.join(output_images_dir, f"{image_name}{ext}")
                        shutil.copy(txt_path, output_txt_path)
                        shutil.copy(image_path, output_image_path)

In [44]:
# Pfade zu den Verzeichnissen
labels_output_dir = "dataset/face_classes/labels"
image_output_dir = "dataset/face_classes/images"
single_dataset_output_labels_dir = "dataset/single_person/labels"
single_dataset_output_images_dir = "dataset/single_person/images"

# Filtere Einzelpersonen vor dem Split
filter_single_persons(labels_output_dir, image_output_dir, single_dataset_output_labels_dir, single_dataset_output_images_dir)

# Train-Test-Split für die Einzelpersonen Dataset
single_dataset_images = [os.path.join(single_dataset_output_images_dir, x) for x in os.listdir(single_dataset_output_images_dir)]
single_dataset_labels = [os.path.join(single_dataset_output_labels_dir, x) for x in os.listdir(single_dataset_output_labels_dir)]

if len(single_dataset_images) < 2 or len(single_dataset_labels) < 2:
    print("Nicht genügend Datenpunkte für den Split vorhanden.")
    exit()

single_train_images, single_val_images, single_train_labels, single_val_labels = train_test_split(
    single_dataset_images, single_dataset_labels, test_size=0.3, random_state=1
)

# Speichern Sie die gesplitteten Daten in den entsprechenden Ordnern
for split, images, labels in zip(['train', 'val'],
                                [single_train_images, single_val_images],
                                [single_train_labels, single_val_labels]):
    split_images_output_dir = os.path.join(single_dataset_output_images_dir, split)
    os.makedirs(split_images_output_dir, exist_ok=True)
    for image_path in images:
        image_filename = os.path.basename(image_path)
        shutil.move(image_path, os.path.join(split_images_output_dir, image_filename))

    split_labels_output_dir = os.path.join(single_dataset_output_labels_dir, split)
    os.makedirs(split_labels_output_dir, exist_ok=True)
    for label_path in labels:
        label_filename = os.path.basename(label_path)
        shutil.move(label_path, os.path.join(split_labels_output_dir, label_filename))


# Pfad zu deiner classes.txt Datei
classes_txt_path = os.path.join(labels_output_dir, 'train', 'classes.txt')

# Kopiere die classes.txt Datei
shutil.copy(classes_txt_path, os.path.join(single_dataset_output_labels_dir, 'train', 'classes.txt'))
shutil.copy(classes_txt_path, os.path.join(single_dataset_output_labels_dir, 'val', 'classes.txt'))

'dataset/single_person/labels\\val\\classes.txt'

### Dunkler Datensatz

Der dunkle Datensatz wurde durch Augmentation verdunkelt, da in Kaggle keine dunklen Datensätze zu finden sind. Das Ziel besteht darin, ein Modell zu trainieren, das auch im Dunkeln Masken erkennen kann. In diesem Datensatz werden alle Klassen berücksichtigt, da es 3 relevante Klassen gibt. Es ist wichtig zu beachten, dass die Annotationen für diesen Datensatz als XML-Dateien vorliegen und für das YOLO-Training in das YOLO-Format konvertiert werden müssen.

In [45]:
# Funktion zur Konvertierung der XML-Datei ins YOLOv5-Format
def convert_xml_to_yolo(xml_path, image_directory, output_dir, class_mapping):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Öffne XML-Datei
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Holen Sie sich den Dateinamen des Bildes (ohne Erweiterung)
    image_filename_without_extension = os.path.splitext(root.find('filename').text)[0]

    # Bildpfad
    image_path = os.path.join(image_directory, f"{image_filename_without_extension}.png")

    # Überprüfen Sie, ob das Bild vorhanden ist
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return

    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    yolo_annotations = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text

        if class_name not in class_mapping:
            raise ValueError(f"Ungültige Klasse: {class_name}")

        # Klassenindizierung
        class_index = class_mapping[class_name]

        # Bounding Box Koordinaten
        bbox = obj.find('bndbox')
        x_min = float(bbox.find('xmin').text)
        y_min = float(bbox.find('ymin').text)
        x_max = float(bbox.find('xmax').text)
        y_max = float(bbox.find('ymax').text)

        # Normalisierung der Koordinaten
        x_center = (x_min + x_max) / (2 * image_width)
        y_center = (y_min + y_max) / (2 * image_height)
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        yolo_annotation = f"{class_index} {x_center} {y_center} {width} {height}"
        yolo_annotations.append(yolo_annotation)

    output_filename = f"{image_filename_without_extension}.txt"
    output_path = os.path.join(output_dir, output_filename)

    with open(output_path, 'w') as f:
        for annotation in yolo_annotations:
            f.write(annotation + '\n')

In [46]:
# Pfade zu den Daten
xml_directory = "../Datasets/Kaggle Face Mask Detection Full/annotations"
image_directory = "../Datasets/Kaggle Face Mask Detection Full/dark/images"
dark_labels_output_dir = "dataset/dark_dataset/labels"
dark_images_output_dir = "dataset/dark_dataset/images"

# Klassenmapping
class_mapping_dark = {
    "without_mask": 0,
    "with_mask": 1,
    "mask_weared_incorrect": 2,
}

# Erstelle Dark Dataset-Ordner, wenn nicht vorhanden
if not os.path.exists(dark_labels_output_dir):
    os.makedirs(dark_labels_output_dir)
if not os.path.exists(dark_images_output_dir):
    os.makedirs(dark_images_output_dir)

# Liste für Bilder und Annotationen
dark_images = []
dark_labels = []

# Durchlaufe die XML-Dateien
for xml_filename in os.listdir(xml_directory):
    if xml_filename.endswith(".xml"):
        xml_path = os.path.join(xml_directory, xml_filename)

        # Konvertiere XML zu YOLOv5-Format
        convert_xml_to_yolo(xml_path, image_directory, dark_labels_output_dir, class_mapping_dark)

        # Extrahiere Bildname ohne Erweiterung
        image_name = os.path.splitext(xml_filename)[0]

        # Füge Bildpfad zur Liste hinzu
        image_path = os.path.join(image_directory, f"{image_name}.png")
        dark_images.append(image_path)

        # Füge Labelpfad zur Liste hinzu
        label_name = f"{image_name}.txt"
        label_path = os.path.join(dark_labels_output_dir, label_name)
        dark_labels.append(label_path)

# Train-Test-Split
if len(dark_images) < 2:
    print("Nicht genügend Datenpunkte für den Split vorhanden.")
    exit()

dark_train_images, dark_val_images, dark_train_labels, dark_val_labels = train_test_split(
    dark_images, dark_labels, test_size=0.3, random_state=1
)

# Kopiere Bilder und Labels in den Dark Dataset-Ordner für Training
for image_path, label_path in zip(dark_train_images, dark_train_labels):
    image_name = os.path.basename(image_path)
    label_name = os.path.basename(label_path)

    output_image_dir = os.path.join(dark_images_output_dir, "train")
    output_label_dir = os.path.join(dark_labels_output_dir, "train")

    # Überprüfe, ob die Zielordner existieren, und erstelle sie, falls nicht
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)

    output_image_path = os.path.join(output_image_dir, image_name)
    output_label_path = os.path.join(output_label_dir, label_name)

    shutil.copy(image_path, output_image_path)
    shutil.copy(label_path, output_label_path)

# Kopiere Bilder und Labels in den Dark Dataset-Ordner für Validierung
for image_path, label_path in zip(dark_val_images, dark_val_labels):
    image_name = os.path.basename(image_path)
    label_name = os.path.basename(label_path)

    output_image_dir = os.path.join(dark_images_output_dir, "val")
    output_label_dir = os.path.join(dark_labels_output_dir, "val")

    # Überprüfe, ob die Zielordner existieren, und erstelle sie, falls nicht
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)

    output_image_path = os.path.join(output_image_dir, image_name)
    output_label_path = os.path.join(output_label_dir, label_name)

    shutil.copy(image_path, output_image_path)
    shutil.copy(label_path, output_label_path)

# Erstelle classes.txt-Datei, mit allen Klassen und deren Indizierung
classes_file_path = os.path.join(dark_labels_output_dir, "classes.txt")
with open(classes_file_path, 'w') as f:
    for class_name, class_index in class_mapping_dark.items():
        f.write(f"{class_index} {class_name}\n")

# Kopiere classes.txt in den Trainingsordner
output_train_classes_path = os.path.join(dark_labels_output_dir, "train", "classes.txt")
shutil.copy(classes_file_path, output_train_classes_path)

# Kopiere classes.txt in den Validierungsordner
output_val_classes_path = os.path.join(dark_labels_output_dir, "val", "classes.txt")
shutil.copy(classes_file_path, output_val_classes_path)

# Lösche alle .txt-Dateien im labels-Ordner
file_extensions_to_delete = [".txt"]

# Lösche alle .txt-Dateien im labels-Ordner
for filename in os.listdir(dark_labels_output_dir):
    if filename.endswith(tuple(file_extensions_to_delete)):
        file_path = os.path.join(dark_labels_output_dir, filename)
        os.remove(file_path)

#### YAML-Datei für alle Datensätze erstellen
Die YAML-Datei ist eine Konfigurationsdatei für das Training des YOLO-Modells. Diese ist wie folgt aufgebaut:
- `train`: Pfad zum Verzeichnis, das die Trainingsbilder enthält.
- `val`: Pfad zum Verzeichnis mit den Validierungsbildern.
- `nc`: Anzahl der Klassen im Datensatz.
- `names`: Eine Liste der Klassennamen. 
- `patience`: Anzahl der Epochen, die gewartet werden, wenn die Leistung des Modells nicht besser wird, bevor das Training beendet wird. Hierbei kann bei Overfitting das Training automatisch abgebrochen werden.
- `delta`: Schwellenwert, um zu bestimmen, ob die Leistung des Modells sich verbessert hat. Wenn die Verbesserung kleiner als dieser Schwellenwert ist, wird das Training gestoppt. 

In [47]:
def create_data_yaml_file(data_path, train_path, val_path, class_names_file):
    with open(class_names_file, 'r') as f:
        class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

    data_yaml = f"train: {train_path}\n" \
                f"val: {val_path}\n" \
                f"nc: {len(class_names)}\n" \
                f"names: {class_names}\n" \
                f"patience: 5  # Anzahl der Epochen, die gewartet werden, wenn die Leistung nicht besser wird\n" \
                f"delta: 0.0001  # Schwellenwert, um zu bestimmen, ob die Leistung sich verbessert hat\n" 
    
    with open(data_path, 'w') as file:
        file.write(data_yaml)
    
    print(f"Die data.yaml-Datei wurde erfolgreich erstellt: {data_path}")

In [48]:
#YAML-Datei für all_classes
create_data_yaml_file(  "dataset/all_classes/data.yaml",
                        "../dataset/all_classes/images/train/",
                        "../dataset/all_classes/images/val/",
                        "dataset/all_classes/labels/train/classes.txt")

Die data.yaml-Datei wurde erfolgreich erstellt: dataset/all_classes/data.yaml


In [49]:
#YAML-Datei für face_classes
create_data_yaml_file(  "dataset/face_classes/data.yaml",
                        "../dataset/face_classes/images/train/",
                        "../dataset/face_classes/images/val/",
                        "dataset/face_classes/labels/train/classes.txt")

Die data.yaml-Datei wurde erfolgreich erstellt: dataset/face_classes/data.yaml


In [50]:
#YAML-Datei für single_person
create_data_yaml_file(  "dataset/single_person/data.yaml",
                        "../dataset/single_person/images/train/",
                        "../dataset/single_person/images/val/",
                        "dataset/single_person/labels/train/classes.txt")

Die data.yaml-Datei wurde erfolgreich erstellt: dataset/single_person/data.yaml


In [51]:
#YAML-Datei für dark_dataset
create_data_yaml_file(  "dataset/dark_dataset/data.yaml",
                        "../dataset/dark_dataset/images/train/",
                        "../dataset/dark_dataset/images/val/",
                        "dataset/dark_dataset/labels/train/classes.txt")

Die data.yaml-Datei wurde erfolgreich erstellt: dataset/dark_dataset/data.yaml
